# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use('Cairo')  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns

from candas.test import QuantStudio     
from candas.lims import Librarian, library

from tinydb import TinyDB
lib = Librarian().load(library).open()

In [2]:
import pathlib as pl

code_pth = pl.Path.cwd()  # for running in Jupyter
# code_pth = pl.Path(__file__)  # for running in terminal
fig_pth = code_pth.parent
data_pth = fig_pth / 'data'
graph_pth = fig_pth / 'graphics'
graph_pth.mkdir(exist_ok=True)

# Figure formatting

In [4]:
fig_num = 6
plt.style.use('style.mplstyle')

%config InlineBackend.figure_format = 'retina'

panel = None
subpanel = None

def savefig(fig=None, title=None):
    fig = fig or plt.gcf()
    title = title or 'Fig_'+'_'.join([str(el) for el in [fig_num, panel, subpanel] if el is not None])
    print(f'Saving {title} in {graph_pth}:', end='')
    for ext in ['svg', 'png']:
        print(f' [.{ext}]', end='')
        fig.savefig(graph_pth / f'{title}.{ext}', dpi=600, transparent=True)
    print(' done')

In [5]:
width = 3.45
height = 2.75
figsize = (width, height)
spotsize = 4**2
ticklabelsize = 5
labelsize = 6
linewidth = 1

palette = sns.diverging_palette(20, 220, as_cmap=True)

mar_l = 0.3
mar_r = 0.3
mar_t = 0.22
mar_b = 0.275
ax_space = 0.075

def format_sns_axes(g, xticks, xlim, ylim=None):
    fig = g.figure
    fig.set_size_inches(figsize)
    
    # g.set_titles('')

    ylim = ylim or [-1.1, 1.1]

    plt.setp(
        g.axes,
        ylim=ylim,
        yticks=[-1, -0.5, 0, 0.5, 1],
        xticks=xticks,
        xlim=xlim,
        title="",
    )
    for i, ax in enumerate(g.axes.flat):
        ax.tick_params(axis="both", labelsize=ticklabelsize, length=1, width=0.6)
        ax.set_ylabel(ax.get_ylabel(), fontsize=labelsize, labelpad=0)
        ax.set_xlabel(ax.get_xlabel(), fontsize=labelsize, labelpad=2)
        # if i > 0:
        #     ax.yaxis.set_tick_params(length=0)
        ax.axhline(0, color="k", linestyle="-", linewidth=0.5, zorder=-1)


    for (row, col), ax in g.axes_dict.items():
        ax.set_title('')
            
        if col==8.0:
            ax2 = ax.twinx()
            ax2.set_yticks([])
            ax2.set_ylabel(f'{row:.1f}', va='bottom', fontsize=labelsize, rotation=270, labelpad=1)
            if row==5.0:
                ax2.set_ylabel(f'log$_1$$_0$ Copies Competitor 2\n{row:.1f}', va='bottom', fontsize=labelsize, rotation=270, labelpad=1)
        
        if row==8.0:
            ax.set_title(f'{col:.1f}', fontsize=labelsize, pad=2)
            if col==5.0:
                ax.set_title(f'log$_1$$_0$ Copies Competitor 1\n{col:.1f}', fontsize=labelsize, pad=2)
                
        if ax != g.axes[-1, 0]:
            ax.set_xlabel('')
            ax.set_ylabel('')


    bbox = ax.get_window_extent().transformed(ax.figure.dpi_scale_trans.inverted())
    ax_width, ax_height = bbox.width, bbox.height

    plt.subplots_adjust(
        left=mar_l / width,
        right=1 - mar_r / width,
        top=1 - mar_t / height,
        bottom=mar_b / height,
        wspace=ax_space/ax_width,
        hspace=ax_space/ax_height,
    )

# Import Data

## JG074C - Antiparallel

In [6]:
db_file = data_pth / 'JG074C Reaction Specifications.json'
rxn_db = TinyDB(db_file)
JG074C_rxns = pd.DataFrame([rxn['oligos'] | {'Sample': rxn['name']} for rxn in rxn_db.all()])

cmax = 50

file = data_pth / 'JG074C_L2_500_competitor_sweep.xlsx'
JG074C = (
    QuantStudio(file, 'JG074C')
    .import_data()
    .format_reactions()
    .index_reactions()
    .subtract_background()
    .normalize_reactions(cmax=cmax)  # , method='min-max')
    .invert_fluorophore('HEX')
)

JG074C.reactions.data = JG074C.reactions.data.merge(JG074C_rxns)
for oligo in ['L2_500', 'S037.01.01b', 'S036.1b']:
    JG074C.reactions.data['lg10 ' + oligo] = np.log10(JG074C.reactions.data[oligo])

# One competitor was clearly missing from this mastermix
JG074C.reactions.wide = JG074C.reactions.wide[~(
    (JG074C.reactions.wide['lg10 S037.01.01b'] == 8.0) &
    (JG074C.reactions.wide['lg10 S036.1b'] == 2.0)
)]

JG074C.extract_endpoints(cmax=cmax, name='FAM-HEX');

## JG074D - Parallel

In [7]:
db_file = data_pth / 'JG074D Reaction Specifications.json'
rxn_db = TinyDB(db_file)
JG074D_rxns = pd.DataFrame([rxn['oligos'] | {'Sample': rxn['name']} for rxn in rxn_db.all()])

cmax = 50

file = data_pth / 'JG074D_L2_500_like_competitor_sweep.xlsx'
JG074D = (
    QuantStudio(file, 'JG074D')
    .import_data()
    .format_reactions()
    .index_reactions()
    .subtract_background()
    .normalize_reactions(cmax=cmax)
    .invert_fluorophore('HEX')
)

JG074D.reactions.data = JG074D.reactions.data.merge(JG074D_rxns)
for oligo in ['L2_500_like', 'S037.01.01b', 'S036.0']:
    JG074D.reactions.data['lg10 ' + oligo] = np.log10(JG074D.reactions.data[oligo])

JG074D.extract_endpoints(cmax=cmax, name='FAM-HEX');

# Plotting

## Antiparallel

### Curves

In [8]:
hue = 'lg10 L2_500'
extent = np.max(np.abs(JG074C.reactions.wide[hue]))
norm = mpl.colors.Normalize(vmin=JG074C.reactions.wide[hue].min(), vmax=JG074C.reactions.wide[hue].max())

g = sns.relplot(data = JG074C.reactions.data.query('Cycle<=@cmax'),
                x='Cycle', y='Fluorescence', col='lg10 S037.01.01b', row='lg10 S036.1b', units='Reaction', hue = hue,
                legend = False, palette='ch:0_r', hue_norm=norm, kind='line', estimator=None, linewidth=linewidth,
                row_order = JG074C.reactions.data['lg10 S036.1b'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.set_ylabels(r'$\leftarrow$ HEX    FAM $\rightarrow$')
g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[0, 10, 20, 30, 40, 50], xlim=[-2.5, 52.5])

g.set_yticklabels([1.0, 0.5, 0.0, 0.5, 1.0])

panel = 'A'
savefig()

Saving Fig_6_A in /home/john/projects/CAN_paper/fig_6/graphics: [.svg] [.png] done


### Endpoints

In [9]:
endpoints = JG074C.endpoints

hue = 'FAM-HEX'
extent = np.max(np.abs(endpoints[hue]))
norm = mpl.colors.Normalize(vmin=-extent, vmax=+extent)

g = sns.relplot(data = endpoints[endpoints.Reporter=='FAM'],
                x='lg10 L2_500', y=hue, col='lg10 S037.01.01b', row='lg10 S036.1b',hue=hue, hue_norm=norm,
                legend = False, palette=palette, kind='scatter', s=spotsize,
                row_order = endpoints['lg10 S036.1b'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[2, 4, 6, 8], xlim=[0.5, 8.5])
g.axes[-1, 0].set_xlabel('log$_1$$_0$ Copies Target')

panel = 'B'
savefig()

Saving Fig_6_B in /home/john/projects/CAN_paper/fig_6/graphics: [.svg] [.png] done


## Parallel

### Curves

In [10]:
hue = 'lg10 L2_500_like'
extent = np.max(np.abs(JG074D.reactions.wide[hue]))
norm = mpl.colors.Normalize(vmin=JG074D.reactions.wide[hue].min(), vmax=JG074D.reactions.wide[hue].max())

g = sns.relplot(data = JG074D.reactions.data.query('Cycle<=@cmax'),
                x='Cycle', y='Fluorescence', col='lg10 S037.01.01b', row='lg10 S036.0', units='Reaction', hue = hue,
                legend = False, palette='ch:0_r', hue_norm=norm, kind='line', estimator=None, linewidth=linewidth,
                row_order = JG074D.reactions.data['lg10 S036.0'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.set_ylabels(r'$\leftarrow$ HEX    FAM $\rightarrow$')
g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[0, 10, 20, 30, 40, 50], xlim=[-2.5, 52.5])

g.set_yticklabels([1.0, 0.5, 0.0, 0.5, 1.0])

panel = 'C'
savefig()

Saving Fig_6_C in /home/john/projects/CAN_paper/fig_6/graphics: [.svg] [.png] done


### Endpoints

In [11]:
endpoints = JG074D.endpoints

hue = 'FAM-HEX'
extent = np.max(np.abs(endpoints[hue]))
norm = mpl.colors.Normalize(vmin=-extent, vmax=+extent)

g = sns.relplot(data = endpoints[endpoints.Reporter=='FAM'],
                x='lg10 L2_500_like', y=hue, col='lg10 S037.01.01b', row='lg10 S036.0', hue=hue, hue_norm=norm,
                legend = False, palette=palette, kind='scatter', s=spotsize,
                row_order = endpoints['lg10 S036.0'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[2, 4, 6, 8], xlim=[0.5, 8.5])
g.axes[-1, 0].set_xlabel('log$_1$$_0$ Copies Target')

panel = 'D'
savefig()

Saving Fig_6_D in /home/john/projects/CAN_paper/fig_6/graphics: [.svg] [.png] done
